In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [3]:
portfolio = pd.read_excel(io="Workshop Data.xlsx",sheet_name="Portfolio")
TenYearYield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="10yUST Yields")
HYyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="HY Index")
IGyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="IG Index")
MoveIdx = pd.read_excel(io="MOVE Vix prices.xlsx",sheet_name="MOVE Index")
Vix = pd.read_excel(io="MOVE Vix prices.xlsx",sheet_name="VIX")

portfolio = portfolio.set_index("Date")
TenYearYield = TenYearYield.set_index("Date").sort_index()
HYyield = HYyield.set_index("Date").sort_index()
IGyield = IGyield.set_index("Date").sort_index()
MoveIdx = MoveIdx.set_index("Date").sort_index()
Vix = Vix.set_index("Date").sort_index()

portfolio.loc[:,"SPYCumDiv"]=portfolio.loc[:,"SPY Dividends"][::-1].cumsum()[::-1]
portfolio.loc[:,"TotalValueSPY"]=portfolio.loc[:,"SPY Position"]+portfolio.loc[:,"SPYCumDiv"]
portfolio = portfolio.sort_index()

SPYCorrData = pd.DataFrame(portfolio.loc["2021-01-11":"2025-12-31","TotalValueSPY"]).merge(HYyield.loc["2021-01-11":"2025-12-31","OAS_SOVEREIGN_CURVE"],left_index=True,right_index=True)
SPYCorrData.loc[:,"TotalReturnsSPY"] = SPYCorrData["TotalValueSPY"].pct_change()
SPYCorrData = SPYCorrData.loc["2021-01-11":"2025-12-31"].merge(HYyield.loc["2021-01-11":"2025-12-31","INDEX_Z_SPREAD_BP"],left_index=True,right_index=True)
SPYCorrData = SPYCorrData.rename(columns={"OAS_SOVEREIGN_CURVE":"HY_Index_OAS","INDEX_Z_SPREAD_BP":"HY_INDEX_Z_SPREAD"})
SPYCorrData.loc[:,"HY_Index_OAS_Diff"]=SPYCorrData["HY_Index_OAS"].diff()
SPYCorrData.loc[:,"HY_INDEX_Z_SPREAD_Diff"]=SPYCorrData["HY_INDEX_Z_SPREAD"].diff()

SPYCorrData = SPYCorrData.loc["2021-01-11":"2025-12-31"].merge(TenYearYield.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
SPYCorrData = SPYCorrData.rename(columns={"PX_LAST":"10y_UST_INDEX_PX"})
SPYCorrData.loc[:,"10yYieldDiff"] = SPYCorrData["10y_UST_INDEX_PX"].diff()

SPYCorrData = SPYCorrData.loc["2021-01-11":"2025-12-31"].merge(IGyield.loc["2021-01-11":"2025-12-31","OAS_SOVEREIGN_CURVE"],left_index=True,right_index=True)
SPYCorrData = SPYCorrData.rename(columns={"OAS_SOVEREIGN_CURVE":"IG_Index_OAS"})
SPYCorrData.loc[:,"IG_Index_OAS_Diff"]=SPYCorrData["IG_Index_OAS"].diff()
SPYCorrData = SPYCorrData.loc["2021-01-11":"2025-12-31"].merge(IGyield.loc["2021-01-11":"2025-12-31","INDEX_Z_SPREAD_BP"],left_index=True,right_index=True)
SPYCorrData = SPYCorrData.rename(columns={"INDEX_Z_SPREAD_BP":"IG_INDEX_Z_SPREAD"})
SPYCorrData.loc[:,"IG_Index_Z_SPREAD_Diff"]=SPYCorrData["IG_INDEX_Z_SPREAD"].diff()

SPYCorrData = SPYCorrData.loc["2021-01-11":"2025-12-31"].merge(MoveIdx.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
SPYCorrData = SPYCorrData.rename(columns={"PX_LAST":"MOVE Idx"})

SPYCorrData = SPYCorrData.loc["2021-01-11":"2025-12-31"].merge(Vix.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
SPYCorrData = SPYCorrData.rename(columns={"PX_LAST":"Vix Idx"})

SPYCorrData=SPYCorrData.dropna()
SPYCorrData = SPYCorrData.sort_index(ascending=False)